In [1]:
%load_ext autoreload
%autoreload 2

from src.cleaner.auto_cleaner import clean_text
import glob
import json
import random

In [2]:
path_list=glob.glob("../data/categorized/**/*.jsonl",recursive=True)


In [3]:
len(path_list)

10384169

In [4]:
random.shuffle(path_list)

In [110]:
line_list=[]
path=random.choice(path_list)
with open(path,"r") as f:
    #1行ずつ読み込む
    for line in f:
        text=json.loads(line)["text"]
        cleaned=clean_text(text)
        print(cleaned)
        break

タイプ男性用利き手右利き商品コード210022558XXXXクラブ種別ドライバーメーカー名テーラーメイド商品名GLOIRE F 11°発売年2014シャフトGLOIRE GL3300(ドライバー)シャフト材質カーボンメーカーフレックスSフレックスSクラブセット組合せ内容クラブセット本数0クラブセット不足本数0クラブセット実本数0総合ランクランクについて傷状態:ヘッド上部C傷状態:フェースC傷状態:ヘッド下部C傷状態:シャフトB傷状態:グリップオリジナル傷あり傷状態:ヘッドカバー純正品(汚い)査定特記・商品状態メーカーカタログ情報となりますので、実商品と異なる場合がございます。
アボダームキャット!.!
三菱マテリアル三菱
こんにちわ。最近、梅干しを1日1個食べることを日課にしているkudoxです。今回は、先日公開した500個▼293-8936カレイナット/M6、板厚1.6ミリ以上、S6-15の使い方を簡単なサンプルと共に紹介したいと思います。BitmapData for EaselJSは、基本的にActionScript3.0に合わせて作っているので、Flashをやられている方は違和感なく使うことができると思います。
PreloadJSは、画像・音声・XML・JS・CSSなどの外部ファイルの読み込みをサポートしてくれるJavaScriptライブラリです。規格(10.2X200)LSD DIY・工具入数(1)からダウンロードできます。以前の投稿「PreloadJSで外部ファイルを読み込む」では、PreloadJS 0.1.0時点の使い方を紹介しましたが、先日公開された新バージョンPreloadJS 0.3.xでは仕様が大幅に変更になりました。そこで新しくなったPreloadJSの変更点や基本的な使い方をレポートしたいと思います。
ひめじやHIMEJIYA


In [19]:
print(clean_text(text))

さあ、働き方をえらぼう
その他のキーワード
以下の派遣のお仕事情報や検索ボックスを選択することで、
その他の派遣のお仕事情報を検索いただくことができます。こちらもご覧ください
▼こちらの条件で求人をお探し下さい
マークの説明
【マイカーOK!
活かせる経験・スキル>Excel
1450円
交通費別途支給
【憧れの大手企業】大手メーカーでの技術者の事務アシスタント
大手メーカー開発部門での技術者アシスタント。技術者の事務作業補助をお願いします。
活かせる経験・スキル>Word、Excel
1250円
交通費別途支給
業界未経験OK!職種未経験OK!
アデコ株式会社
【一部在宅勤務】大手企業での営業事務!
大手メーカーでの営業事務のお仕事です。
活かせる経験・スキル>Excel、英語力を活かす
1320円
交通費別途支給
営業事務経験のある方業界未経験OK!
アデコ株式会社
【一部在宅勤務】大手企業での事務!
大手メーカーでの経理兼営業事務のお仕事です。
活かせる経験・スキル>Excel、英語力を活かす
1320円
交通費別途支給
事務経験のある方業界未経験OK!職種未経験OK!
アデコ株式会社
活かせる経験・スキル>Word、Excel
1300円
交通費別途支給
年金システムの保守業務《COBOL・SQL》
システム、ソフトウェア開発企業にて改修のお仕事。/COBOLとSQLのスキル活かせます!
活かせる経験・スキル>
中京区
1500円
交通費別途支給
【以下のご経験のある方は歓迎いたします!
ハウスメーカでのCAD業務です
駅近!通勤も楽々/CADのご経験を活かしませんか?
活かせる経験・スキル>CAD関連
1300円
交通費別途支給
【以下のご経験のある方は歓迎いたします!】建築知識。CAD経験。
資格なくても大丈夫!
活かせる経験・スキル>Excel、CAD関連
1500円
交通費別途支給
【以下のご経験のある方は歓迎いたします!】建築関連の一般知識。建築士資格(尚可)。
1500円
交通費別途支給
京都三条の駅チカ企業で給与計算など事務業務(パートタイム)
活かせる経験・スキル>Excel、資格を活かす
1100円
【キャリアが浅い、ブランク復帰も大歓迎!
1450円*名駅!大手商社でSAPを使用したデータ入力など
活かせる経験・スキル>
1450円
交通

In [384]:
print(text)

スペースがなくなってしまうので、と、TRMCのオフィスに寄贈してくれました。


In [ ]:
a="川崎市高津区でブラックでも即日融"
b="川崎市高津区でブラックでは即日融"

set(a)-set(b)

In [ ]:
line